In [1]:
# Create instance
# imports
import random
import sys
import re
from graphviz import Graph
import math as math
import os

global max_velocity
global R

R = 350
max_velocity = 17

In [2]:
class Arc:
    def __init__(self):
        self.time_s = 0.0
        self.actual_delay = 0.0
        self.avr_delay = 0.0
        self.cnt_delay = 0
        self.cnt_jitter = 0
        self.avr_jitter = 0.0
        self.bdw = 0.0
        self.start_time = 0.0
        self.stop_time = 0.0
        self.bytes = 0.0
        self.link = 0
        self.link_duration = 0.0
        self.packet_time = 0.0
        self.seq_number = 0
        self.relative_vel = [0.0 for i in range(max_velocity)]
        self.correlation = 0.0
        self.distance = 0.0
        self.L = 0.0
        self.T = 0.0
        self.average = 0.0
        self.deviation = 0.0
        self.variance = 0.0
        self.ldp = 0.0

class Car:
    def __init__(self):
        self.x = 0.0
        self.y = 0.0
        self.population = 0
        self.velocity = 0.0
        self.sum = 0.0
        self.avr = 0.0
        self.deviation = 0.0
        self.distance = 0.0
        self.sum_variance = 0.0
        self.variance = 0.0
        self.direction = ['', '']
        self.module = 0.0
        

In [3]:
def f(t, L, average, variance, deviation):
    equation = 0.0;
    equation = (-1*(((L / (t - average)) * (L / (t - average))) / (2 * variance)))
    return (L / (t * (math.sqrt((2* math.pi * deviation))))) * math.exp(equation)
# h = 0.1
def simpson(h, x0, xn, L, average, variance, deviation):
    i = 0
    n = 1000
    y = [0.0 for i in range(n+1)]
    x = [0.0 for i in range(n+1)]
    so = 0.0
    se = 0.0 
    ans = 0.0
    
    h = (xn - x0)/n
    if(n % 2 == 1):
        n += 1;
    for i in range(n+1):
        x[i] = x0 + i*h
        y[i] = f(x[i], L, average, variance, deviation)
    
    for i in range(n):
        if(i % 2 == 1):
            so += y[i]
        else:
            se += y[i];
            
    ans = h/3 * (y[0]+y[n-1]+4*so+2*se)
    return (ans)

In [4]:
def treat_movement(tokens):
    time = float(tokens[1])
    node = int(tokens[2])
    x_vector_aux = re.findall(r'\d+', tokens[3])
    x = float(x_vector_aux[0] + '.' + x_vector_aux[1]) 
    
    y_vector_aux = re.findall(r'\d+', tokens[4])
    y = float(y_vector_aux[0] + '.' + y_vector_aux[1]) 
    
    if (x - cars[node].x > 0):
        cars[node].direction[0] = 'R'
    else:
        cars[node].direction[0] = 'L'
    cars[node].x = x
    
    if (y - cars[node].y > 0):
        cars[node].direction[1] = 'T'
    else:
        cars[node].direction[1] = 'B'
    cars[node].y = y
    
    speed = float(tokens[8])
    if (speed == 0):
        cars[node].velocity = 0.1
    else:
        cars[node].velocity = speed
    
    velocities[node] = cars[node].velocity
    
    if(cars[node].population == max_velocity-1):
        cars[node].population = 0
    
    velocity_matrix[node][cars[node].population] = cars[node].velocity
    cars[node].population += 1

def link_duration(n):
    # Speed average
    for i in range(n):
        for j in range(max_velocity):
            cars[i].sum += velocity_matrix[i][j]
            if (cars[i].population > max_velocity):
                cars[i].population = max_velocity
        try:
            cars[i].avr = (cars[i].sum / cars[i].population)
        except ZeroDivisionError:
            cars[i].avr = 0
    # Variance and standard deviation
    for i in range(n):
        for j in range(max_velocity):
            cars[i].sum_variance += ((velocity_matrix[i][j] - cars[i].avr)**2) 
        
        try:
            cars[i].variance = (cars[i].sum_variance / (cars[i].population-1))
            cars[i].deviation = math.sqrt(cars[i].variance)
        except ZeroDivisionError:
            cars[i].variance = cars[i].deviation = 0
        
    # Link Duration
    for i in range(n):
        for j in range(n):
            if (arcs[i][j].link == 1):
                cont = 0
                vt = 0
                covar = 0
                for z in range(max_velocity):
                    v1 = 0
                    v2 = 0
                    # Covariancia entre a velocidade dos nós
                    if (velocity_matrix[i][z] != 0 and velocity_matrix[j][z] != 0):
                        v1 = velocity_matrix[i][z] - cars[i].avr
                        v2 = velocity_matrix[j][z] - cars[j].avr
                        arcs[i][j].relative_vel[z] = abs(velocity_matrix[i][z] - velocity_matrix[j][z])
                        cont += 1
                        vt = v1 * v2
                        covar += vt
                try:        
                    arcs[i][j].correlation = ((covar / cont) / (cars[i].deviation * cars[j].deviation))
                except ZeroDivisionError:
                    arcs[i][j].correlation = 0
                
                arcs[i][j].distance = math.sqrt(abs(((cars[i].x - cars[j].x) * (cars[j].x - cars[i].x)) + ((cars[i].y - cars[j].y) * (cars[j].y - cars[i].y))))
                
#                 if (arcs[i][j].distance > R):
#                     arcs[i][j].link = 0
#                     arcs[j][i].link = 0
                
                # Vehicles on the same direction
                if (cars[i].direction[0] == cars[j].direction[0] and cars[i].direction[1] == cars[j].direction[1]):
                    cars[i].module = (cars[i].x**2) * (cars[i].y**2)
                    cars[i].module = (cars[j].x**2) * (cars[j].y**2)
                    
                    # Verifing the direction
                    if (cars[i].direction[1] == "T"):
                        # How vehicle is the first
                        if (cars[i].module > cars[j].module):
                            if (velocities[i] > velocities[j]):
                                arcs[i][j].L = R - arcs[i][j].distance
                                
                                if (abs(velocities[i] - velocities[j]) == 0):
                                    arcs[i][j].T = arcs[i][j].L / 0.1
                                else:
                                    try:
                                        arcs[i][j].T = arcs[i][j].L / abs(velocities[i] - velocities[j])
                                    except ZeroDivisionError:
                                        arcs[i][j].T = 0
                            else:
                                arcs[i][j].L = R + arcs[i][j].distance
                                
                                if (abs(velocities[i] - velocities[j]) == 0):
                                    arcs[i][j].T = arcs[i][j].L / 0.1
                                else:
                                    try:
                                        arcs[i][j].T = arcs[i][j].L / abs(velocities[i] - velocities[j])
                                    except ZeroDivisionError:
                                        arcs[i][j].T = 0
                        else:
                            if (velocities[i] > velocities[j]):
                                arcs[i][j].L = R + arcs[i][j].distance
                                
                                if (abs(velocities[i] - velocities[j]) == 0):
                                    arcs[i][j].T = arcs[i][j].L / 0.1
                                else:
                                    try:
                                        arcs[i][j].T = arcs[i][j].L / abs(velocities[i] - velocities[j])
                                    except ZeroDivisionError:
                                        arcs[i][j].T = 0
                            else:
                                arcs[i][j].L = R - arcs[i][j].distance
                                
                                if (abs(velocities[i] - velocities[j]) == 0):
                                    arcs[i][j].T = arcs[i][j].L / 0.1
                                else:
                                    try:
                                        arcs[i][j].T = arcs[i][j].L / abs(velocities[i] - velocities[j])
                                    except ZeroDivisionError:
                                        arcs[i][j].T = 0
                    # Vehicles to bottom
                    else:
                        # How vehicle is the first
                        if (cars[i].module < cars[j].module):
                            if (velocities[i] > velocities[j]):
                                arcs[i][j].L = R - arcs[i][j].distance
    
                                if (abs(velocities[i] - velocities[j]) == 0):
                                    arcs[i][j].T = arcs[i][j].L / 0.1
                                else:
                                    arcs[i][j].T = arcs[i][j].L / abs(velocities[i] - velocities[j])
                            else:
                                arcs[i][j].L = (2 * R) + arcs[i][j].distance
                                
                                if (abs(velocities[i] - velocities[j]) == 0):
                                    arcs[i][j].T = arcs[i][j].L / 0.1
                                else:
                                    arcs[i][j].T = arcs[i][j].L / abs(velocities[i] - velocities[j])
                        else:
                            if (velocities[i] > velocities[j]):
                                arcs[i][j].L = (2 * R) + arcs[i][j].distance
                                
                                if (abs(velocities[i] - velocities[j]) == 0):
                                    arcs[i][j].T = arcs[i][j].L / 0.1
                                else:
                                    arcs[i][j].T = arcs[i][j].L / abs(velocities[i] - velocities[j])
                            else:
                                arcs[i][j].L = R - arcs[i][j].distance
                                
                                if (abs(velocities[i] - velocities[j]) == 0):
                                    arcs[i][j].T = arcs[i][j].L / 0.1
                                else:
                                    arcs[i][j].T = arcs[i][j].L / abs(velocities[i] - velocities[j])
                # Both vehicles from the same side, but onde is to Top othe to the bottom
                elif (cars[i].direction[0] == cars[j].direction[0] and cars[i].direction[1] != cars[j].direction[1]):
                    if (cars[i].direction[1] == "T"):
                        if (cars[i].y > cars[j].y):
                            arcs[i][j].L = R - arcs[i][j].distance
                        else:
                            arcs[i][j].L = R + arcs[i][j].distance
                        try:
                            arcs[i][j].T = arcs[i][j].L / abs(velocities[i] - velocities[j])
                        except ZeroDivisionError:
                            arcs[i][j].T = 0
                    else:
                        if (cars[i].y > cars[j].y):
                            arcs[i][j].L = R + arcs[i][j].distance
                        else:
                            arcs[i][j].L = R - arcs[i][j].distance
                        try:
                            arcs[i][j].T = arcs[i][j].L / abs(velocities[i] - velocities[j])
                        except ZeroDivisionError:
                            arcs[i][j].T = 0
                # Both vehicles on direction top/Bottom and the vehicle i to the right            
                elif (cars[i].direction[0] != cars[j].direction[0] and cars[i].direction[1] == cars[j].direction[1]):
                    if (cars[i].direction[0] == "R"):
                        if (cars[i].x > cars[j].x):
                            arcs[i][j].L = R - arcs[i][j].distance
                        else:
                            arcs[i][j].L = R + arcs[i][j].distance
                        try:
                            arcs[i][j].T = arcs[i][j].L / abs(velocities[i] - velocities[j])
                        except ZeroDivisionError:
                            arcs[i][j].T = 0
                    else:
                        if (cars[i].x > cars[j].x):
                            arcs[i][j].L = R + arcs[i][j].distance
                        else:
                            arcs[i][j].L = R - arcs[i][j].distance
                        try:
                            arcs[i][j].T = arcs[i][j].L / abs(velocities[i] - velocities[j])
                        except ZeroDivisionError:
                            arcs[i][j].T = 0
                else:
                    if (cars[i].direction[1] == "T" and cars[i].y > cars[j].y):
                        arcs[i][j].L = R - arcs[i][j].distance
                    elif (cars[i].direction[1] == "T" and cars[i].y <= cars[j].y):
                        arcs[i][j].L = R + arcs[i][j].distance
                    elif (cars[i].direction[1] == "B" and cars[i].y > cars[j].y):
                        arcs[i][j].L = R + arcs[i][j].distance
                    elif (cars[i].direction[1] == "B" and cars[i].y <= cars[j].y):
                        arcs[i][j].L = R - arcs[i][j].distance
                    try:
                        arcs[i][j].T = arcs[i][j].L / abs(velocities[i] - velocities[j])
                    except ZeroDivisionError:
                        arcs[i][j].T = 0
    for i in range(n):
        for j in range(n):
            if (arcs[i][j].link == 1):
                vt = 0
                cont = 0
                for z in range(max_velocity):
                    if (arcs[i][j].relative_vel[z] != 0):
                        vt += arcs[i][j].relative_vel[z]
                        cont += 1
                
                try:
                    arcs[i][j].average = vt/cont
                except ZeroDivisionError:
                    arcs[i][j].average = 0
      
    for i in range(n):
        for j in range(n):
            if (arcs[i][j].link == 1):
                vt = 0
                cont = 0
                for z in range(max_velocity):
                    if (arcs[i][j].relative_vel[z] != 0):
                        vt += ((arcs[i][j].relative_vel[z] - arcs[i][j].average)**2)
                        cont += 1
                
                try:
                    arcs[i][j].variance = vt/(cont-1)
                except ZeroDivisionError:
                    arcs[i][j].variance = 0
                try:
                    arcs[i][j].deviation = math.sqrt(arcs[i][j].variance)
                except ZeroDivisionError:
                    arcs[i][j].deviation = 0
                
                lim_sup = 0.0
                lim_inf = 0.0
                
                try:
                    lim_sup = arcs[i][j].L / (arcs[i][j].average - (4 * arcs[i][j].deviation))
                except ZeroDivisionError:
                    lim_sup = 0.0
                try:
                    lim_inf = arcs[i][j].L / (arcs[i][j].average + (4 * arcs[i][j].deviation))
                except ZeroDivisionError:
                    lim_inf = 0.0
                
                average = arcs[i][j].average
                variance = arcs[i][j].variance
                L = arcs[i][j].L
                deviation = arcs[i][j].deviation
        
                try:
                    arcs[i][j].ldp = simpson(0.1, lim_inf, lim_sup, L, average, variance, deviation)
                except ZeroDivisionError:
                    arcs[i][j].ldp = 0

In [5]:
global num_of_terminals
def send(tokens):
    global source
    global send_time
    global seq_number
    
    if (len(tokens) > 20):
        msg_type = tokens[20][1:]
        if (tokens[3] == "MAC" and msg_type == "HELLO"):
            send_time = float(tokens[1])
            source = int(re.findall(r'\d+', tokens[2])[0])
            seq_number = int(tokens[6])

def received(tokens):
    global source
    global send_time
    global seq_number

    if (len(tokens) > 20):
        received_time = float(tokens[1])
        destine = int(re.findall(r'\d+', tokens[2])[0])
        seq_number_dest = int(tokens[6])
        len_pkg = int(tokens[8])
        msg_type = tokens[20][1:]
        if (tokens[3] == "MAC" and msg_type == "HELLO" and seq_number == seq_number_dest):
            arcs[source][destine].link = arcs[source][destine].link = 1
            # Calculating Delay
            delay = (received_time - send_time)
            arcs[source][destine].avr_delay += delay
            arcs[source][destine].cnt_delay += 1

            packet_interval = (10.0 + random.randrange(500))
            packet_interval = packet_interval/100000
            arcs[source][destine].packet_time += (delay + packet_interval)   

            # Calculating Jitter
            if (arcs[source][destine].actual_delay == 0.0):
                arcs[source][destine].actual_delay = delay
                arcs[source][destine].cnt_jitter += 1
            else:
                jitter = abs(float(arcs[source][destine].actual_delay - float(delay)))
                arcs[source][destine].avr_jitter += jitter
                arcs[source][destine].cnt_jitter += 1
                arcs[source][destine].actual_delay = delay

            # Calculating Bandwith
            arcs[source][destine].bytes += float(len_pkg)
            
def trace_extract(trace_file, interval, nodes, isDurationLink):
    global source
    global send_time
    global seq_number

    source = 0
    send_time = 0.0
    seq_number = 0
    
    if (isDurationLink):
        for line in trace_file:
            tokens = line.split(" ")
            if (float(tokens[1]) <= interval[1]):
                if (tokens[0] == "M"):
                    if (int(re.findall(r'\d+', tokens[2])[0]) < nodes):
                        treat_movement(tokens)
            else:
                break
    else:
        for line in trace_file:
            tokens = line.split(" ")
            if (len(tokens) > 20):
                if (interval[0] <= float(tokens[1]) <= interval[1]):
                    if (int(re.findall(r'\d+', tokens[2])[0]) < nodes):
                        # Handing with the sending packets
                        if (tokens[0] == "s"):
                            send(tokens)
                        # Handing with the received packets
                        elif (tokens[0] == "r"):
                            received(tokens)
                else:
                    break

def generate_instance(trace_file_name, instance_name, n, interval):    
    global arcs
    global cars
    global max_velocity
    global velocity_matrix
    global velocities
    global num_of_terminals

    trace_file = None
    flow_file = None
    root = random.randint(1, n)
    edges = 0
    cnt_terminals = int(0.2*n) + random.randint(0, int(0.15*n))
    num_of_terminals = cnt_terminals
    terminals = []
    graph = Graph(name=instance_name + '.png', format='png')
    
    arcs = [[Arc() for j in range(n)] for i in range(n)]
    cars = [Car() for i in range(n)]
    velocities = [0 for i in range(n)]
    velocity_matrix = [[0.0 for j in range(max_velocity)] for i in range(n)]
    
    try:
        trace_file = open(trace_file_name, 'r')
    except:
        print("Cannot open the file, check the path or the name of instance!")
        sys.exit(0)
    
    instance = open(instance_name + "-" + str(cnt_terminals) + '.txt', 'w')
    
    trace_extract(trace_file, interval, n, False)
    trace_file.seek(0, 0)
    trace_extract(trace_file, interval, n, True)
    link_duration(n)
    
    i = 0
    while i < cnt_terminals:
        new_terminal = random.randint(1, n)
        if (new_terminal not in terminals and new_terminal != root):
            terminals.append(new_terminal)
            i += 1
    
    # Draw Graph
    for i in range(n):
        graph.node(name = str(i+1), shape = 'circle')
    
    graph.node(name = str(root), color = 'green')
    
    for t in terminals:
        graph.node(name = str(t), color = 'red')
    
    for i in range(n):
        for j in range(n):
            if (arcs[i][j].link == 1 and arcs[j][i].link == 1):
                try:
                    arcs[i][j].avr_delay = (arcs[i][j].avr_delay/(arcs[i][j].cnt_delay/10.6))
                except ZeroDivisionError:
                    arcs[i][j].avr_delay = 0.0
                try:
                    arcs[i][j].avr_jitter = (arcs[i][j].avr_jitter/arcs[i][j].cnt_jitter) * 10.6
                except ZeroDivisionError:
                    arcs[i][j].avr_jitter = 0.0
                try:
                    arcs[i][j].bytes = (arcs[i][j].bytes/arcs[i][j].packet_time) * (0.008)
                except ZeroDivisionError:
                    arcs[i][j].bytes = 0.0                
            else:
                arcs[i][j].link = arcs[j][i].link = 0
    
    edges2write = []
    for i in range(n):            
        for j in range(i+1, n):
            i_m = min(i, j)
            j_m = max(i, j)

            max_delay = max(arcs[i][j].avr_delay, arcs[j][i].avr_delay)

            max_jitter = max(arcs[i][j].avr_jitter, arcs[j][i].avr_jitter)

            max_bytes = max(arcs[i][j].bytes, arcs[j][i].bytes)

            max_ldp = max(arcs[i][j].ldp, arcs[j][i].ldp)

            if (max_ldp > 0):
                edges += 1
                graph.edge(str(i+1), str(j+1))
                edges2write.append("E " + str(i+1) + " " + str(j+1) + " " + str(format(max_delay, '.6f')) + " " + str(format(max_jitter, '.6f')) + " " + str(format(max_bytes, '.2f')) + " " + str(format(max_ldp, '.2f')) + "\n")

    
    # Write the instance on .txt file
    instance.write('SECTION Comment\n')
    instance.write('CREATOR \"Carlos Araujo\"\n')
    instance.write('REMARK \"Instance for QoS Multicast Routing Problem\"\n')
    instance.write('QoS METRICS \"delay, jitter, bandwidth and estimate of link duration\"\n') 
    instance.write('END\n\n')
    
    instance.write('SECTION Graph\n')
    instance.write('Nodes ' + str(n) + '\n')
    instance.write('Edges ' + str(edges) + '\n\n')
    
    
    for e in edges2write:
        instance.write(e)
    instance.write('END\n\n')
    
    instance.write('SECTION Terminals\n')
    instance.write('Root ' + str(root) + '\n')
    instance.write('Terminals ' + str(len(terminals)) + '\n')
    for t in terminals:
        instance.write('T ' + str(t) + '\n')
    instance.write('END\n\nEOF')
    
    #graph.render()
    
    

In [7]:
nodes = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
size = [50, 75, 100]
for s in size:
    path = "/home/carlos/Documents/Washington-" + str(s)
    os.mkdir(path)
    for n in nodes:
        inst_path = path + "/" + str(n)
        os.mkdir(inst_path)
        trace_file = "/home/carlos/Documents/washington-tracefile-" + str(s) +"-100.tr"
        final_instance = inst_path + "/washington-" + str(s) + "-" + str(n)
        generate_instance(trace_file, final_instance, n, [0, n+20])

In [9]:
nodes = [80, 90, 100]
size = [50, 75, 100]
for s in size:
    path = "/home/carlos/Documents/Washington-" + str(s)
    for n in nodes:
        inst_path = path + "/" + str(n)
        trace_file = "/home/carlos/Documents/washington-tracefile-" + str(s) +"-100.tr"
        final_instance = inst_path + "/1-washington-" + str(s) + "-" + str(n)
        generate_instance(trace_file, final_instance, n, [0, n+50])

In [ ]:
nodes = 10

while (nodes < 20):
    interval = [0, 25]
    for i in range(10, 20, 10):
        tr = '/home/carlos/Documentos/washington-tracefile-100-50.tr'# + str(nodes) + '.tr'
        instance_file = '/home/carlos/Documentos/final-washington-100-' + str(nodes) + '-' + str(i) + '.txt'
        generate_instance(tr, instance_file, nodes,  interval)
        interval[0] = interval[1]
        interval[1] += 15
    nodes += 10

nodes = [15, 25, 50, 75, 100, 125, 150]